In [2]:
!pip install pandas
!pip install numpy
!pip install matplotlib

     --------------------------------------- 10.3/10.3 MB 21.8 MB/s eta 0:00:00
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
     --------------------------------------- 14.8/14.8 MB 32.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 7.6/7.6 MB 19.5 MB/s eta 0:00:00
     ---------------------------------------- 163.0/163.0 kB ? eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ---------------------------------------- 1.0/1.0 MB 21.1 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB ? eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 26.2 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import io, time, json
import requests
from bs4 import BeautifulSoup

In [27]:
def describe(df):
    print('\n*** First 5 rows')
    print(df.head())
    print('\n*** Size and data type')
    size = len(df)
    print(f'Size: {df.size}')
    print(df.dtypes)
    print('\n*** NaN count each column')
    nan_df = df.isna().sum().to_frame(name='count')
    nan_df['percent'] = nan_df['count'] / size * 100
    print(nan_df)
    print('\n*** Value count each column')
    for col in df:
        print('**************************')
        val_df = df[col].value_counts().to_frame(name='count')
        val_df['percent'] = val_df['count'] / size * 100
        print(val_df)

In [28]:
df = pd.read_csv('crime_reduced.csv')
describe(df)


*** First 5 rows
         ID         Primary Type                              Description  \
0  12014684                THEFT                           $500 AND UNDER   
1  11864018   DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT $300 AND UNDER   
2  11859805                THEFT                             RETAIL THEFT   
3  12012127  MOTOR VEHICLE THEFT                               AUTOMOBILE   
4  11863808                THEFT                                OVER $500   

           Location Description  Domestic  Year                    Date  
0                        STREET     False  2020  03/17/2020 09:30:00 PM  
1  COMMERCIAL / BUSINESS OFFICE     False  2019  09/24/2019 08:00:00 AM  
2            GROCERY FOOD STORE     False  2019  10/13/2019 08:30:00 PM  
3                     APARTMENT      True  2020  03/18/2020 02:03:00 AM  
4                     RESIDENCE     False  2019  10/05/2019 06:30:00 PM  

*** Size and data type
Size: 6704054
ID                       int64
Primar

In [26]:
# column Location Description has very few row with NaN value
# we can drop that
df1 = df[~df['Location Description'].isna()]
describe(df1)


*** First 5 rows
         ID         Primary Type                              Description  \
0  12014684                THEFT                           $500 AND UNDER   
1  11864018   DECEPTIVE PRACTICE  FINANCIAL IDENTITY THEFT $300 AND UNDER   
2  11859805                THEFT                             RETAIL THEFT   
3  12012127  MOTOR VEHICLE THEFT                               AUTOMOBILE   
4  11863808                THEFT                                OVER $500   

           Location Description  Domestic  Year                    Date  
0                        STREET     False  2020  03/17/2020 09:30:00 PM  
1  COMMERCIAL / BUSINESS OFFICE     False  2019  09/24/2019 08:00:00 AM  
2            GROCERY FOOD STORE     False  2019  10/13/2019 08:30:00 PM  
3                     APARTMENT      True  2020  03/18/2020 02:03:00 AM  
4                     RESIDENCE     False  2019  10/05/2019 06:30:00 PM  

*** Size and data type
Size: 6673877
ID                       int64
Primar